# 학습데이터, 테스트데이터 만들기

In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
train_ft = pd.read_excel('train_tmp.xlsx')
test_ft = pd.read_excel('test_tmp.xlsx')

train_ft.shape,test_ft.shape

((4112, 8), (1028, 8))

In [3]:
train_texture = train_ft.loc[:,['리뷰','재질_긍부정']].copy()
train_texture = train_texture[train_texture['재질_긍부정'].isin([-1,1])]

test_texture = test_ft.loc[:,['리뷰','재질_긍부정']].copy()
test_texture = test_texture[test_texture['재질_긍부정'].isin([-1,1])]

train_texture.shape,test_texture.shape

((1152, 2), (288, 2))

In [4]:
train_texture['재질_긍부정'].value_counts() 
# 1의 비율이 훨씬 더 높은 수준임을 알 수 있음 

재질_긍부정
 1    878
-1    274
Name: count, dtype: int64

In [5]:
train_texture.columns

Index(['리뷰', '재질_긍부정'], dtype='object')

In [6]:
train_arr = train_texture['리뷰'].to_numpy()
test_arr = test_texture['리뷰'].to_numpy()

train_arr.shape,test_arr.shape

((1152,), (288,))

In [7]:
target = train_texture['재질_긍부정'].to_numpy().reshape(-1,1)
target = np.where(target == -1,0,target)

target_test = test_texture['재질_긍부정'].to_numpy().reshape(-1,1)
target_test = np.where(target_test == -1,0,target_test)

target = target.astype('int64')
target_test = target_test.astype('int64')

target.shape,target_test.shape

((1152, 1), (288, 1))

In [8]:
target.mean()

0.7621527777777778

# 사전학습 모델 선정

In [8]:
model_name = 'kykim/bert-kor-base'

In [9]:
from transformers import AutoTokenizer, AutoModel

In [10]:
model = AutoModel.from_pretrained(model_name)

/home/lab01/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 사전학습모델을 바탕으로 학습 실행

In [12]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, x, y=None):
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        item = {}
        item["x"] = self.get_tokenizer(self.x[idx])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[idx])
        return item
    def get_tokenizer(self, text):
        x = self.tokenizer(text, padding="max_length", truncation=True)
        for k, v in x.items():
            x[k] = torch.tensor(v)
        return x

In [13]:
dt = ReviewDataset(tokenizer, train_arr, target)
dl = torch.utils.data.DataLoader(dt, batch_size=2, shuffle=False)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    2, 30231,  8048,  ...,     0,     0,     0],
         [    2, 14932,  8506,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 'y': tensor([[1.],
         [1.]])}

In [14]:
class Net(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.pre_model = AutoModel.from_pretrained(model_name)
        self.fc_out = torch.nn.Linear( self.pre_model.config.hidden_size, 1)

    def forward(self, x):
        x = self.pre_model(**x)
        # x[0]: 모든 시점의 히든출력 batch, seq, features
        # x[1]: CLS 토큰의 히든출력 batch, features
        return self.fc_out(x[1])

In [15]:
model = Net(model_name)
model(batch["x"])

tensor([[-0.1458],
        [ 0.0017]], grad_fn=<AddmmBackward0>)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train() # 학습 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        loss = loss_fn( pred, batch["y"].to(device) )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)
    return epoch_loss

In [17]:
@torch.no_grad()
def test_loop(dataloader, model, loss_fn, device):
    epoch_loss = 0
    pred_list = []
    act_func = torch.nn.Sigmoid()
    model.eval() # 평가 모드
    for batch in tqdm(dataloader):
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn( pred, batch["y"].to(device) )
            epoch_loss += loss.item()

        pred = act_func(pred) # logit 값을 확률로 변환
        pred = pred.to("cpu").numpy() # cpu 이동후 ndarray 로변환
        pred_list.append(pred)

    epoch_loss /= len(dataloader)
    pred = np.concatenate(pred_list)
    return epoch_loss, pred

In [18]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

n_splits = 5
cv = KFold(n_splits, shuffle=True, random_state=42)

batch_size = 8 # 배치 사이즈
loss_fn = torch.nn.BCEWithLogitsLoss() # 손실 객체
epochs = 100 # 최대 가능한 에폭수

In [19]:
is_holdout = True
reset_seeds(42) # 재현을 위해 시드고정
best_score_list = []
for i, (tri, vai) in enumerate( cv.split(train_arr) ):
    # 학습용 데이터로더 객체
    train_dt = ReviewDataset(tokenizer, train_arr[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)

    # 검증용 데이터로더 객체
    valid_dt = ReviewDataset(tokenizer, train_arr[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체와 옵티마이저 객체 생성
    model = Net(model_name).to(device)
    optimizer = torch.optim.Adam( model.parameters(), lr=2e-5 )

    best_score = 0 # 현재 최고 점수
    patience = 0 # 조기 종료 조건을 주기 위한 변수
    for epoch in range(epochs):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)

        pred = (pred > 0.5).astype(int) # 이진분류 문제에서 클래스 번호 결정
        score = f1_score(target[vai], pred,average = 'macro')

        print(train_loss, valid_loss, score)
        if score > best_score:
            best_score = score # 최고 점수 업데이트
            patience = 0
            torch.save(model.state_dict(), f"texture_{i}.pth") # 최고 점수 모델 가중치 저장

        patience += 1
        if patience == 5:
            break

    print(f"{i}번째 폴드 최고 정확도: {best_score}")
    best_score_list.append(best_score)

    if is_holdout:
        break

/home/lab01/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


  0%|          | 0/116 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

0.43517267790719355 0.27861295576239453 0.8787878787878788


  0%|          | 0/116 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

0.018709663187799944 0.8353757703373739 0.8354978354978355
0번째 폴드 최고 정확도: 0.922077922077922


/home/lab01/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


  0%|          | 0/116 [00:00<?, ?it/s]

KeyboardInterrupt: 

# 테스트데이터 학습 

In [ ]:
test_dt = ReviewDataset(tokenizer,test_arr)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=2, shuffle=False)

In [ ]:
pred_list = []
for i in range(n_splits):
    model = Net(model_name).to(device)
    state_dict = torch.load(f"texture_{i}.pth")
    model.load_state_dict(state_dict)

    _, pred = test_loop(test_dl, model, loss_fn, device)

    pred_list.append(pred)
    if is_holdout:
        break

In [ ]:
pred = np.mean(pred_list, axis=0)
pred = (pred > 0.5).astype(int)

In [ ]:
score = f1_score(target_test, pred,average = 'macro')
score